In [1]:
import math
import numpy as np 
import pandas as pd 
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Bidirectional, Dropout, Conv1D, Lambda, SimpleRNN
import matplotlib.pyplot as plt 
import pandas_datareader as web
import statsmodels.api as sm  
plt.style.use('fivethirtyeight')

In [4]:
from os.path import dirname
dirPath = dirname(os.getcwd())+"\codec\TotalCleanedTest.csv"
dirPath

'c:\\Users\\David Fernandez\\Desktop\\cancerDataAnalysis\\codec\\TotalCleanedTest.csv'

In [6]:
df = pd.read_csv(dirPath)
df['ent_regis'] = df['ent_regis'].apply(lambda x: '{0:0>2}'.format(x))
df['ent_resid'] = df['ent_resid'].apply(lambda x: '{0:0>2}'.format(x))
df['ent_ocurr'] = df['ent_ocurr'].apply(lambda x: '{0:0>2}'.format(x))
df = df.drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
df.head()

,ent_regis,ent_regis_nm,mun_regis,mun_regis_nm,ent_resid,ent_resid_nm,mun_resid,mun_resid_nm,ent_ocurr,ent_ocurr_nm,...,complicaro,lengua,ICD,detailCaus,Organ,ICD_Mex,detailCaus_Mex,organ_Mex,date_ocurr,month_year_ocurr
0,01,Aguascalientes,1005,Jesus Maria,01,Aguascalientes,1005,Jesus Maria,01,Aguascalientes,...,8,9,C34,"Tumor maligno de los bronquios o del pulmón, p...",Lungs,C34,"Tumor maligno de la tráquea, de los bronquios ...",Lungs,2012-03-06,2012-03-01
1,01,Aguascalientes,1011,San Francisco de los Romo,01,Aguascalientes,1011,San Francisco de los Romo,01,Aguascalientes,...,8,9,C71,"Tumor maligno del encéfalo, parte no especificada",Brain,C71,Tumor maligno del encéfalo,Brain,2012-02-25,2012-02-01
2,01,Aguascalientes,1003,Calvillo,01,Aguascalientes,1001,Aguascalientes,01,Aguascalientes,...,8,9,C67,"Tumor maligno de la vejiga urinaria, parte no ...",Bladder,C67,Tumor maligno de la vejiga urinaria,Bladder,2012-04-28,2012-04-01
3,01,Aguascalientes,1001,Aguascalientes,01,Aguascalientes,1001,Aguascalientes,01,Aguascalientes,...,8,9,C56,Tumor maligno del ovario,Ovary,C56,Tumor maligno del ovario,Ovary,2012-07-26,2012-07-01
4,01,Aguascalientes,1001,Aguascalientes,01,Aguascalientes,1001,Aguascalientes,01,Aguascalientes,...,8,9,C25,"Tumor maligno del páncreas, parte no especificada",Pancreas,C25,Tumor maligno del páncreas,Pancreas,2012-01-11,2012-01-01


In [74]:
#population dictionary
popInfo = pd.read_csv("https://dl.dropboxusercontent.com/s/ihun7h2mkhwgell/population.csv?dl=0")
popInfo.set_index('year', inplace=True)
popDict = popInfo.to_dict()['population']

In [81]:
def generate_report(year, age_range):
    dff = df
    dff = dff.loc[(dff['edad'] >= age_range[0]) & (dff['edad'] <= age_range[1]) & (dff['anio_ocur'] == year)]
    curr_count = dff['ICD'].value_counts().rename_axis('unique_values').reset_index(name='counts')
    col_one_list = curr_count['unique_values'].tolist()
    output = list()
    for i in col_one_list:
        add_row = dff.loc[(dff['ICD'] == i) & (dff['sexo'] == 1)]
        add_row = add_row['ICD'].value_counts()
        if len(add_row.values) > 0:
            output.append(add_row.values[0])
        else:
            output.append(0)
    curr_count['male'] = output
    cols = ['counts', 'male']
    curr_count['female'] = curr_count['counts'] - curr_count['male']
    ndir = dirname(os.getcwd())
    pPath = dirname(os.getcwd()) + '\\codec\\types.xlsx'
    currDict = pd.read_excel(pPath)
    currDict.set_index('ICD', inplace=True)
    organ_dict = currDict.to_dict()['organ']
    curr_count['Organ'] = curr_count['unique_values'].map(organ_dict)
    curr_count['Rate'] = round((curr_count['counts']/popDict.get(2012))*100000, 3)
    cols = df.columns.tolist()
    cols = ['unique_values', 'Organ', 'male', 'female', 'counts', 'Rate']
    curr_count = curr_count[cols]
    curr_count.columns = ['ICD', 'Organ', 'Male Cases', 'Female Cases', 'Total Cases', 'Death Rate per 100,000']
    return curr_count

In [82]:
count = generate_report(2012, [50, 60])
count

,ICD,Organ,Male Cases,Female Cases,Total Cases,"Death Rate per 100,000"
0,C50,Breast,8,1682,1690,1.441
1,C53,Cervix Uteri,0,1214,1214,1.035
2,C25,Pancreas,578,572,1150,0.980
3,C16,Stomach,555,511,1066,0.909
4,C34,Lungs,632,420,1052,0.897
5,C20,Colon and Rectum,525,439,964,0.822
6,C22,Liver,398,410,808,0.689
7,C96,Lymphoma,307,320,627,0.535
8,C56,Ovary,0,566,566,0.483
9,C64,Kidney,305,176,481,0.410
